<a href="https://colab.research.google.com/github/100479095/Predictor_F1_2025/blob/main/Entrenamiento_Modelo_F1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time

#URL de la API de Ergast
BASE_URL = "http://ergast.com/api/f1"

def get_race_data(season):
    races_url = f"{BASE_URL}/{season}.json"
    r = requests.get(races_url)
    data = r.json()

    #Obtenemos la lista de carreras de esa temporada
    races = data['MRData']['RaceTable']['Races']
    season_data = []

    for race in races:
        round_num = int(race['round'])
        race_name = race['raceName']

        #Obtenemos los resultados de cada carrera (Grid y Posición Final)
        res_url = f"{BASE_URL}/{season}/{round_num}/results.json"
        res_req = requests.get(res_url)
        try:
            res_data = res_req.json()['MRData']['RaceTable']['Races'][0]['Results']
        except (IndexError, KeyError):
            continue #Si no hay resultados (carrera futura o cancelada), la saltamos

        # Obtener STANDINGS *antes* de la carrera
        # Si es la ronda 1, todos tienen 0 puntos. Si no, pedimos la ronda anterior.
        driver_standings_map = {}
        const_standings_map = {}

        if round_num > 1:
            #Truco: Pedimos los standings de la ronda anterior
            st_url = f"{BASE_URL}/{season}/{round_num - 1}/driverStandings.json"
            st_req = requests.get(st_url)
            try:
                st_list = st_req.json()['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']
                for s in st_list:
                    driver_id = s['Driver']['driverId']
                    driver_standings_map[driver_id] = int(s['position'])
            except:
                pass

            # Lo mismo para constructores (para simplificar, usaremos solo driver en este ejemplo)

        # 3. Construir el dataset fila por fila
        for result in res_data:
            driver_id = result['Driver']['driverId']
            grid = int(result['grid'])
            try:
                position = int(result['position']) # Posición final
            except ValueError:
                position = 20 # DNF u otros, lo penalizamos como último para el modelo

            # Si es la ronda 1, asumimos posición de standings basada en el año anterior o aleatoria
            # Para simplificar el modelo: si no hay dato (ronda 1), ponemos posición 10 (media)
            standings_pos = driver_standings_map.get(driver_id, 10)

            # Guardar datos anonimizados (sin nombres)
            season_data.append({
                'season': season,
                'round': round_num,
                'driver_standings_pos': standings_pos,
                'grid_position': grid,
                'finishing_position': position
                # Aquí podrías añadir 'constructor_standings_pos' si haces la llamada extra
            })

        # Respetar los límites de la API (evitar bloqueo)
        time.sleep(0.5)

    return pd.DataFrame(season_data)

# --- EJECUCIÓN ---
# Recopilamos datos desde 2020 hasta 2023 (entrenamiento)
years = [2021, 2022, 2023, 2024]
all_data = pd.DataFrame()

for y in years:
    df_season = get_race_data(y)
    all_data = pd.concat([all_data, df_season], ignore_index=True)

print("Datos descargados. Guardando en CSV...")
all_data.to_csv('data/f1_training_data.csv', index=False)
print(all_data.head())